# Credit Risk Model

### Business Problem

Bank XYZ is in the business of giving personal loans, structured as [non-recourse loans](http://www.investopedia.com/terms/n/nonrecoursedebt.asp). The defaults on their loans are much higher than their competitors. Also, the underlying collaterals lose their value way too quicky and has resulted in huge losses for Bank XYZ.

Alice was recently appointed as the Senior VP of the Risk Organization. She comes from a strong analytics background and wants to leverage data science to identify customer's risk before approving loan.

She's appointed you as a consultant to help her and the team solve this problem.


*Note: This case study was inspired by the [bank marketing case study](https://archive.ics.uci.edu/ml/datasets/bank+marketing). The data is a modified version of what is available in that site*

**Brainstorming**

### 1. Frame

The first step is to convert the business problem into an analytics problem

Alice wants to know customer's risk. Let's try to predict the propensity of a customer to default, given the details he/she has entered on the loan application form

### 2. Acquire

After discussions with the IT team of Bank XYZ, you have obtained some historical data from the bank. It has the following columns

**Application Attributes**:
- `years`: Number of years the applicant has been employed  
- `ownership`: Whether the applicant owns a house or not  
- `income`:  Annual income of the applicant  
- `age`: Age of the applicant  
- `amount` : Amount of Loan requested by the applicant  

**Behavioural Attributes**:
- `grade`:  Credit grade of the applicant

**Outcome Variable**:

- `default` : Whether the applicant has defaulted or not 

#### Load the data

In [ ]:
#Load the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#Defualt Variables
%matplotlib inline
plt.rcParams['figure.figsize'] = (16,9)
plt.style.use('fivethirtyeight')
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [ ]:
#Load the training dataset
df = pd.read_csv("../data/historical_loan.csv")

In [ ]:
#View the first few rows of training dataset
df.head()

In [ ]:
#View the columns of the train dataset
df.columns

In [ ]:
#View the data types of the train dataset
df.dtypes

In [ ]:
#View the number of records in the data
df.shape

In [ ]:
#View summary of raw data 
df.describe()

### 2. Refine

Lets check the dataset for compeleteness - by checking for missing values


**Missing values**

In [ ]:
# Find if df has missing values.
df.isnull()

In [ ]:
# In a large dataset, this is hard to find if there are any missing values or not.
# We can chain operators on the output. Let's use sum()

df.isnull().sum()

So, we see that `years` have missing values. The column is numeric. We have three options for dealing with missing values

**Options to treat Missing Values**

1. **REMOVE** - NAN rows
2. **IMPUTATION** - Replace them with something??
     - Mean
     - Median
     - Fixed Number - Domain Relevant
     - High Number (999) - Issue with modelling
3. **BINNING** - Categorical variable and "Missing becomes a number
4. **DOMAIN SPECIFIC** - Entry error, pipeline, etc.

In [ ]:
# Let's replace missing values with mean
# There's a fillna function
df.years = df.years.fillna(np.mean(df.years))

In [ ]:
# Now, check if training dataframe has missing values or not
<your code goes here>

In [ ]:
#Finding unique values of years 
pd.unique(df.years)

We also need to check for quality - by checking for outliers in the data. For this workshop, we will skip doing that. But remember to check for outliers when doing in real-life

### 3. Explore

The goal is to build some intuition around the data

** Single Variable Exploration - Univariate Analysis**

In [ ]:
# Create histogram for target variable - default


In [ ]:
# Explore amount


** Dual Variable Exploration - Bivariate Analysis**

In [ ]:
# Create a crosstab of grade and ownership


In [ ]:
# Explore the impact of age with default


** Three Variables Exploration - Bivariate Analysis**

In [ ]:
# Explore the relationship between age, income and default


### 4. Transform

In [ ]:
# Let's again revisit the data types in the dataset
df.dtypes

Two of the columns are categorical in nature - grade and ownership.

To build models, we need all of the features to be numeric. There exists a number of ways to convert categorical variables to numeric values.

We will use one of the popular options: `LabelEncoding`

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
# Let's not modify the original dataset. 
# Let's transform it in another dataset
df_encoded = df.copy()

In [ ]:
# instantiate label encoder
le_grade = LabelEncoder()

In [ ]:
# fit label encoder
le_grade = le_grade.fit(df_encoded["grade"])

In [ ]:
df_encoded.grade = le_grade.transform(df_encoded.grade)

In [ ]:
df_encoded.head()

**Exercise**

Do label encoding on ownership

In [ ]:
<Write your code here>


In [ ]:
#le_ownership = LabelEncoder()
#le_ownership = le_ownership.fit(df["ownership"])
#df_encoded.ownership = le_ownership.transform(df_encoded.ownership)
#df_encoded.head()

### 5. Model

Common approaches:

1. Linear models
2. Tree-based models
3. Bayesian Models
4. Artificial Neural Networks

Some choices to consider:

1. Interpretability
2. Run-time
3. Model complexity
4. Scalability

For the purpose of this workshop, we will use tree-based models.

We will do the following two:

1. Decision Tree
2. Random Forest

### Decision Trees

Decision Trees are a non-parametric supervised learning method used for classification and regression. The goal is to create a model that predicts the value of a target variable by learning simple decision rules inferred from the data features.

Let's first build a model using just three features to build some intuition around decision trees

**Step 1 -** Create features matrix and target vector

In [ ]:
X = df_encoded.loc[:,('age', 'income', 'years')]
y = df_encoded.loc[:,'default']

** Step 2 -** Build decision tree model

In [ ]:
from sklearn import tree

In [ ]:
# instantiate the decision tree object
clf = tree.DecisionTreeClassifier(max_depth=2)

In [ ]:
# fit the decision tree model
clf = clf.fit(X, y)

**Step 3 -** Visualize the decision tree

In [ ]:
import pydotplus 
from IPython.display import Image

In [ ]:
dot_data = tree.export_graphviz(clf, out_file='tree_3.dot', feature_names=X.columns,
                                class_names=['no', 'yes'], filled=True, 
                                rounded=True, special_characters=True)

In [ ]:
graph = pydotplus.graph_from_dot_file('tree_3.dot')  

In [ ]:
Image(graph.create_png()) 

In addition, it makes sense to visualize the decision boundaries, for all the variables - and even make a small animation to see the boundaries in 3-D.

This will outside the scope of this workshop. But the related code for this can be found in the speakers' introductory workshop on machine learning [here](https://github.com/amitkaps/applied-machine-learning/blob/master/Module-03b-Model-Trees.ipynb)

### Model Validation

While we have created the model, we still don't have a *measure* of how good the model is. We need to measure some accuracy metric of the model and have confidence that it will generalize well. We should be confident that when we put the model in production (real-life), the accuracy we get from the model results should mirror the metrics we obtained when we built the model.

Selecting the right accuracy metric for the model is important. 

[This wiki](https://en.wikipedia.org/wiki/Evaluation_of_binary_classifiers) has a good overview of some of the common metrics.

We will use a metric - **Area Under the Curve**

#### Area Under the Curve

In a Receiver Operating Characteristic (ROC) curve the true positive rate (Sensitivity) is plotted in function of the false positive rate (100-Specificity) for different cut-off points. Each point on the ROC curve represents a sensitivity/specificity pair corresponding to a particular decision threshold. A test with perfect discrimination (no overlap in the two distributions) has a ROC curve that passes through the upper left corner (100% sensitivity, 100% specificity). Therefore the closer the ROC curve is to the upper left corner, the higher the overall accuracy of the test 
([source](https://www.medcalc.org/manual/roc-curves.php))


![](img/roc.png)

### Cross-validation

Now that we have chosen the error metric, how do we find the generalization error?

We do this using cross-validation. ([source]
(https://en.wikipedia.org/wiki/Cross-validation_(statistics))

From wiki: 
> One round of cross-validation involves partitioning a sample of data into complementary subsets, performing the analysis on one subset (called the training set), and validating the analysis on the other subset (called the validation set or testing set). To reduce variability, multiple rounds of cross-validation are performed using different partitions, and the validation results are averaged over the rounds.

![](img/cv.jpg)



We will use `StratifiedKFold`.

This ensures that in each fold, the proportion of positive class and negative class remain similar to the original dataset

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

In [ ]:
# Instantiate stratified k fold. Let's use 3 fold
kf = StratifiedKFold(n_splits=3)

In [ ]:
# Let's use an array to store the results of cross-validation
kfold_auc_score = []

This is the process we will follow to get the mean cv-score

1. Generate k-fold
2. Train the model using k-1 fold
3. Predict for the kth fold 
4. Find the accuracy.
5. Append it to the array
6. Repeat 2-5 for different validation folds
7. Report the mean cross validation score

In [ ]:
# Run kfold CV

for train_index, test_index in kf.split(df_encoded.iloc[:,1:], 
                                        df_encoded.iloc[:,0]):
    clf = tree.DecisionTreeClassifier(max_depth=2)
    clf = clf.fit(df_encoded.iloc[train_index,1:], 
                  df_encoded.iloc[train_index,0])
    dt_prediction = clf.predict_proba(df_encoded.iloc[test_index,1:])
    auc_score = roc_auc_score(df_encoded.iloc[test_index,0],
                            dt_prediction[:,1])
    print(auc_score)
    kfold_auc_score.append(auc_score)

print("Mean K Fold CV:", np.mean(kfold_auc_score))

**Exercise**

Run 5-fold CV and check if the mean cv-score increases or decreases

In [ ]:
<Your code here>

### Bagging

Decision trees in general have low bias and high variance. We can think about it like this: given a training set, we can keep asking questions until we are able to distinguish between ALL examples in the data set. We could keep asking questions until there is only a single example in each leaf. Since this allows us to correctly classify all elements in the training set, the tree is unbiased. However, there are many possible trees that could distinguish between all elements, which means higher variance.

### How do we reduce variance?
In order to reduce the variance of a single error tree, we usually place a restriction on the number of questions asked in a tree. This is true  for single decision trees which we have seen in previous notebooks.

Along with this other method to do reduce variance is to **ensemble models** of decision trees. The goal of ensemble methods is to combine the predictions of several base estimators built with a given learning algorithm in order to improve generalizability / robustness over a single estimator.

### How to ensemble?

1. **Averaging**: Build several estimators independently and then average their predictions. On average, the combined estimator is usually better than any of the single base estimator because its variance is reduced. Examples:
    - Bagging
    - Random Forest
    - Extremely Randomized Trees
 
2. **Boosting**: Build base estimators sequentially and then try to reduce the bias of the combined estimator. The motivation is to combine several weak models to produce a powerful ensemble.
    - AdaBoost
    - Gradient Boosting (e.g. xgboost)
    
### Random Forest

In random forests, each tree in the ensemble is built from a **sample drawn with replacement** (i.e., a bootstrap sample) from the training set. In addition, when splitting a node during the construction of the tree, the split that is chosen is no longer the best split among all features. Instead, the split that is picked is the best split among a **random subset of the features**. 

As a result of this randomness, the bias of the forest usually slightly increases (with respect to the bias of a single non-random tree) but, due to averaging, its variance also decreases, usually more than compensating for the increase in bias, hence yielding an overall better model.



**Random Forest Model**

The advantage of the `scikit-learn` API is that the syntax remains fairly consistent across all the classifiers.

If we change the DecisionTreeClassifier to RandomForestClassifier in the above code, we should be good to go :-)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Instantiate stratified k fold. Let's use 3 fold
kf = StratifiedKFold(n_splits=3)

# Let's use an array to store the results of cross-validation
kfold_auc_score = []

# Run kfold CV

for train_index, test_index in kf.split(df_encoded.iloc[:,1:], 
                                        df_encoded.iloc[:,0]):
    #The only line that needs change is instantiating the right classifier
    clf = RandomForestClassifier(n_estimators=10)
    #End of change
    clf = clf.fit(df_encoded.iloc[train_index,1:], 
                  df_encoded.iloc[train_index,0])
    dt_prediction = clf.predict_proba(df_encoded.iloc[test_index,1:])
    auc_score = roc_auc_score(df_encoded.iloc[test_index,0],
                            dt_prediction[:,1])
    print(auc_score)
    kfold_auc_score.append(auc_score)

print("Mean K Fold CV:", np.mean(kfold_auc_score))

**Exercise**

Change the number of trees from 10 to 100 and make it 5-fold. And report the cross-validation error (Hint: You should get ~ -.73. )

In [ ]:
<your code here>

A more detailed version of bagging and random forest can be found in the speakers' introductory machine learning workshop material

[bagging](https://github.com/amitkaps/applied-machine-learning/blob/master/Module-03d-Model-Bagging.ipynb)  
[random forest](https://github.com/amitkaps/applied-machine-learning/blob/master/Module-03e-Model-RandomForest.ipynb)

### Model Selection

We choose the model and its hyper-parameters that has the best cross-validation score on the chosen error metric.

In our case, it is random forest.

Now - how do we get the model?

We need to run the model with the chosen hyper-parameters on all of the train data. And serialize it.

In [ ]:
final_model = RandomForestClassifier(n_estimators=100)
final_model = final_model.fit(df_encoded.iloc[:,1:], 
                  df_encoded.iloc[:,0])

### Model serialization

We need to serialize the model and the label encoders. 

In [ ]:
from sklearn.externals import joblib

In [ ]:
joblib.dump(final_model, "credit_risk_model.pkl")
joblib.dump(le_grade, "le_grade.pkl")
joblib.dump(le_ownership, "le_ownership.pkl")